## Loading Northwind database from Postgres to Katana Graph

In [ ]:
import os
import psycopg2
import pandas as pd
from katana import remote
from katana.remote import import_data
import dask.dataframe as dd
os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"

In [ ]:
graph = remote.Client().create_graph(
    num_partitions=3
)

In [ ]:
conn = psycopg2.connect(
    host="katana-db-1",
    database="test",
    user="postgres",
    password="abc123")

In [ ]:
print(conn)

![schema](northwind_schema.png)

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM categories")

In [ ]:
categories = pd.DataFrame(cur.fetchall())

In [ ]:
categories = categories.rename(columns={0:"category_id",1:"category_name",2:"description",3:"picture"})

In [ ]:
cur.execute("SELECT * FROM customers")

In [ ]:
customers = pd.DataFrame(cur.fetchall())

In [ ]:
customers = customers.rename(columns={0:"customer_id",1:"company_name",2:"contact_name",
                                      3:"contact_title",4:"address",5:"city",6:"region",
                                      7:"postal_code",8:"country",9:"phone",10:"fax"})

In [ ]:
cur.execute("SELECT * FROM customer_customer_demo")

In [ ]:
customer_customer_demo = pd.DataFrame(cur.fetchall())

In [ ]:
customer_customer_demo = customer_customer_demo.rename(columns={0:"customer_id",1:"customer_type_id"})

In [ ]:
cur.execute("SELECT * FROM customer_demographics")

In [ ]:
customer_demographics = pd.DataFrame(cur.fetchall())

In [ ]:
customer_demographics = customer_demographics.rename(columns={0:"customer_type_id",1:"customer_desc"})

In [ ]:
cur.execute("SELECT * FROM employees")

In [ ]:
employees = pd.DataFrame(cur.fetchall())

In [ ]:
employees = employees.rename(columns={0:"employee_id",1:"last_name",2:"first_name",
                                      3:"contact_title",4:"title",5:"title_of_courtesy",
                                      6:"birth_date",7:"hire_date",8:"address",9:"city",10:"region",
                                      11:"postal_code",12:"country",13:"home_phone",14:"extension",
                                      15:"photo",16:"notes",17:"reports_to",18:"photo_path"})

In [ ]:
cur.execute("SELECT * FROM employee_territories")

In [ ]:
employee_territories = pd.DataFrame(cur.fetchall())

In [ ]:
employee_territories = employee_territories.rename(columns={0:"employee_id",1:"territory_id"})

In [ ]:
cur.execute("SELECT * FROM order_details")

In [ ]:
order_details = pd.DataFrame(cur.fetchall())

In [ ]:
order_details = order_details.rename(columns={0:"order_id",1:"product_id",2:"unit_price",
                                              3:"quantity",4:"discount"})

In [ ]:
cur.execute("SELECT * FROM orders")

In [ ]:
orders = pd.DataFrame(cur.fetchall())

In [ ]:
orders = orders.rename(columns={0:"order_id",1:"customer_id",2:"employee_id",
                                3:"order_date",4:"required_date",5:"shipped_date",
                                6:"ship_via",7:"freight",8:"ship_name",9:"ship_address",10:"ship_city",
                                11:"ship_region",12:"ship_postal_code",13:"ship_country"})

In [ ]:
cur.execute("SELECT * FROM products")

In [ ]:
products = pd.DataFrame(cur.fetchall())

In [ ]:
products = products.rename(columns={0:"product_id",1:"product_name",2:"supplier_id",
                                    3:"category_id",4:"quantity_per_unit",5:"unit_price",
                                    6:"units_in_stock",7:"units_on_order",8:"reorder_level",9:"discontinued"})

In [ ]:
cur.execute("SELECT * FROM region")

In [ ]:
region = pd.DataFrame(cur.fetchall())

In [ ]:
region = region.rename(columns={0:"region_id",1:"region_description"})

In [ ]:
cur.execute("SELECT * FROM shippers")

In [ ]:
shippers = pd.DataFrame(cur.fetchall())

In [ ]:
shippers = shippers.rename(columns={0:"shipper_id",1:"company_name",2:"phone"})

In [ ]:
cur.execute("SELECT * FROM suppliers")

In [ ]:
suppliers = pd.DataFrame(cur.fetchall())

In [ ]:
suppliers = suppliers.rename(columns={0:"supplier_id",1:"company_name",2:"contact_name",
                                      3:"contact_title",4:"address",5:"city",
                                      6:"region",7:"postal_code",8:"country",9:"phone",
                                      10:"fax",11:"homepage"})

In [ ]:
cur.execute("SELECT * FROM territories")

In [ ]:
territories = pd.DataFrame(cur.fetchall())

In [ ]:
territories = territories.rename(columns={0:"territory_id",1:"territory_description",2:"region_id"})

In [ ]:
cur.execute("SELECT * FROM us_states")

In [ ]:
us_states = pd.DataFrame(cur.fetchall())

In [ ]:
us_states = us_states.rename(columns={0:"state_id",1:"state_name",2:"state_abbr",3:"state_region"})

In [ ]:
has_product_category = pd.merge(products[['product_id', 'category_id']], categories['category_id'], how="left", on="category_id")

In [ ]:
cur.execute("SELECT p.product_id AS product_id, c.category_id AS category_id FROM products p LEFT JOIN categories c ON p.category_id = c.category_id")

In [ ]:
product_categories = pd.DataFrame(cur.fetchall())

In [ ]:
product_categories = product_categories.rename(columns={0:"product_id",1:"category_id"})

In [ ]:
%%time
with import_data.DataFrameImporter(graph) as df_importer:   
    
    df_importer.nodes_dataframe(categories,
                            id_column="category_id",
                            id_space="Category",
                            label="Category")
    
    df_importer.nodes_dataframe(products,
                            id_column="product_id",
                            id_space="Product",
                            label="Product")
    
    df_importer.nodes_dataframe(orders,
                            id_column="order_id",
                            id_space="Order",
                            label="Order")
    
    df_importer.nodes_dataframe(customers,
                            id_column="customer_id",
                            id_space="Customer",
                            label="Customer")

    df_importer.nodes_dataframe(suppliers,
                            id_column="supplier_id",
                            id_space="Supplier",
                            label="Supplier")
    
    df_importer.nodes_dataframe(order_details,
                            id_column="order_id",
                            id_space="OrderDetail",
                            label="OrderDetail")
    
    df_importer.nodes_dataframe(employees,
                            id_column="employee_id",
                            id_space="Employee",
                            label="Employee")
    
    df_importer.nodes_dataframe(shippers,
                            id_column="shipper_id",
                            id_space="Shipper",
                            label="Shipper")
    
    df_importer.nodes_dataframe(employee_territories,
                            id_column="employee_id",
                            id_space="EmployeeTerritory",
                            label="EmployeeTerritory")
    
    df_importer.nodes_dataframe(territories,
                            id_column="territory_id",
                            id_space="Territory",
                            label="Territory")
    
    df_importer.nodes_dataframe(region,
                            id_column="region_id",
                            id_space="Region",
                            label="Region")
    
    df_importer.nodes_dataframe(us_states,
                            id_column="state_id",
                            id_space="State",
                            label="State")

    df_importer.edges_dataframe(product_categories[['product_id', 'category_id']],
                            source_id_space="Category",
                            destination_id_space="Product",
                            source_column="category_id",
                            destination_column="product_id",
                            type="HAS_PRODUCT_CATEGORY")

In [ ]:
%%time
graph.query('''MATCH (c:Customer), (o:Order {customer_id:c.id}) 
               CREATE (c)-[:HAS_ORDER]->(o)''')

In [ ]:
%%time
graph.query('''MATCH (od:OrderDetail), (o:Order {id:od.id}) 
               CREATE (o)-[:HAS_ORDER_DETAIL]->(od)''')

In [ ]:
%%time
graph.query('''MATCH (od:OrderDetail), (p:Product)
               WHERE toString(od.product_id) = p.id
               CREATE (od)-[:HAS_PRODUCT]->(p)''')

In [ ]:
%%time
graph.query('''MATCH (s:Supplier), (p:Product) 
               WHERE toString(p.supplier_id) = s.id
               CREATE (p)-[:HAS_SUPPLIER]->(s)''')

In [ ]:
%%time
graph.query('''MATCH (e:Employee), (o:Order) 
               WHERE toString(o.employee_id) = e.id
               CREATE (e)-[:ATTACHED_TO]->(o)''')

In [ ]:
%%time
graph.query('''MATCH (e:Employee), (et:EmployeeTerritory {id:e.id}) 
               CREATE (e)-[:IN_TERRITORY]->(et)''')

In [ ]:
%%time
graph.query('''MATCH (et:EmployeeTerritory), (t:Territory) 
               WHERE toString(et.territory_id) = t.id
               CREATE (t)-[:HAS_TERRITORY]->(et)''')

In [ ]:
%%time
graph.query('''MATCH (s:Shipper), (o:Order) 
               WHERE toString(o.ship_via) = s.id
               CREATE (o)-[:HAS_SHIPPER]->(s)''')

In [ ]:
graph.query('MATCH (n)-[r]->(m) RETURN type(r) AS reltype, count(r) AS numrels, labels(n) AS orig, labels(m) AS term ORDER by numrels DESC', contextualize=True).view()

In [ ]:
%%time
graph.query('''MATCH (t:Territory), (r:Region)
               WHERE toString(t.region_id) = r.id
               CREATE (t)-[:HAS_REGION]->(r)''')

In [ ]:
graph.query('MATCH (n) RETURN n', contextualize=True).view()